# 📊 EDA — Film Insights Lab

Notebook de **Exploração de Dados (EDA)** para o dataset de filmes.
- Caminho padrão do CSV: `data/movies.csv`
- Gráficos usando **matplotlib** (sem seaborn), um gráfico por célula.
- **Objetivos**: qualidade de dados, estatísticas, correlações, distribuição de variáveis, texto (Overview) e insights rápidos.

> Gerado automaticamente em 2025-09-10.

In [ ]:
# %% [setup]
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

CSV_PATH = "data/movies.csv"  # ajuste se necessário
assert os.path.exists(CSV_PATH), f"Não encontrei o arquivo: {CSV_PATH}"
df_raw = pd.read_csv(CSV_PATH)
df_raw.head()

## 1. Visão Geral do Dataset

In [ ]:
# Dimensões, colunas, dtypes
print("Shape:", df_raw.shape)
display(pd.DataFrame({
    "coluna": df_raw.columns,
    "dtype": [str(t) for t in df_raw.dtypes],
}).reset_index(drop=True))

df_raw.sample(min(5, len(df_raw)), random_state=42)

## 2. Valores Ausentes (Missing)

In [ ]:
missing = df_raw.isna().sum().sort_values(ascending=False)
missing_pct = (missing / len(df_raw) * 100).round(2)
miss_df = pd.DataFrame({"missing": missing, "missing_%": missing_pct})
display(miss_df)

# Gráfico de barras dos missing
plt.figure()
miss_df["missing_%"].plot(kind="bar")
plt.title("Percentual de Missing por Coluna")
plt.xlabel("Colunas")
plt.ylabel("% Missing")
plt.tight_layout()
plt.show()

## 3. Duplicatas

In [ ]:
dups = df_raw.duplicated().sum()
print(f"Duplicatas (linhas idênticas): {dups}")

## 4. Padronização mínima para análises

In [ ]:
import re

def _to_minutes(runtime):
    if pd.isna(runtime): return None
    m = re.search(r"(\d+)", str(runtime))
    return int(m.group(1)) if m else None

def _to_year(s):
    if pd.isna(s): return None
    m = re.search(r"(\d{4})", str(s))
    return int(m.group(1)) if m else None

def _to_gross_number(g):
    if pd.isna(g): return None
    s = str(g).strip().replace("$", "").replace(",", "")
    try:
        return float(s)
    except ValueError:
        return None

df = df_raw.copy()
# renomeia se houver colunas usuais do IMDb Top 1000
rename_map = {
    "Series_Title": "Title",
    "Released_Year": "Year",
    "Runtime": "Runtime",
    "Genre": "Genre",
    "IMDB_Rating": "IMDB_Rating",
    "Overview": "Overview",
    "Meta_score": "Meta_score",
    "Director": "Director",
    "No_of_Votes": "No_of_Votes",
    "Gross": "Gross",
    "Certificate": "Certificate",
}
df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

if "Runtime" in df.columns:
    df["Runtime_min"] = df["Runtime"].apply(_to_minutes)
if "Year" in df.columns:
    df["Year_num"] = df["Year"].apply(_to_year)
if "Gross" in df.columns:
    df["Gross_clean"] = df["Gross"].apply(_to_gross_number)
if "Genre" in df.columns:
    df["Genre_main"] = df["Genre"].astype(str).apply(lambda s: s.split(",")[0].strip() if s and s!="nan" else None)
if "Overview" in df.columns:
    df["Overview"] = df["Overview"].astype(str).fillna("")
df.head()

## 5. Estatísticas Descritivas

In [ ]:
num_cols = [c for c in ["Runtime_min","Year_num","IMDB_Rating","Meta_score","No_of_Votes","Gross_clean"] if c in df.columns]
display(df[num_cols].describe().T)

## 6. Correlações (numéricas)

In [ ]:
if len(num_cols) >= 2:
    corr = df[num_cols].corr(numeric_only=True)
    display(corr)

    # Heatmap simples com matplotlib (sem seaborn)
    plt.figure()
    im = plt.imshow(corr.values, interpolation="nearest")
    plt.title("Matriz de Correlações")
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=45, ha="right")
    plt.yticks(range(len(corr.index)), corr.index)
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.tight_layout()
    plt.show()
else:
    print("Poucas colunas numéricas para correlação.")

## 7. Distribuições univariadas

In [ ]:
for col in num_cols:
    plt.figure()
    df[col].dropna().hist(bins=30)
    plt.title(f"Distribuição de {col}")
    plt.xlabel(col)
    plt.ylabel("Frequência")
    plt.tight_layout()
    plt.show()

## 8. Categóricas — contagens

In [ ]:
for col in ["Genre_main", "Certificate", "Director"]:
    if col in df.columns:
        vc = df[col].value_counts().head(20)
        display(vc.to_frame(name="count"))
        plt.figure()
        vc.plot(kind="bar")
        plt.title(f"Top categorias — {col}")
        plt.xlabel(col)
        plt.ylabel("Contagem")
        plt.tight_layout()
        plt.show()

## 9. Texto (Overview) — tamanho e exemplos

In [ ]:
if "Overview" in df.columns:
    lens = df["Overview"].astype(str).apply(len)
    plt.figure()
    lens.hist(bins=30)
    plt.title("Distribuição do tamanho do Overview (n caracteres)")
    plt.xlabel("n caracteres")
    plt.ylabel("Frequência")
    plt.tight_layout()
    plt.show()

    # Exemplos
    display(df[["Title","Overview"]].dropna().head(5))
else:
    print("Coluna 'Overview' não encontrada.")

## 10. Relações interessantes (dispersões)

In [ ]:
pairs = [
    ("No_of_Votes", "IMDB_Rating"),
    ("Meta_score", "IMDB_Rating"),
    ("Year_num", "IMDB_Rating"),
    ("Runtime_min", "IMDB_Rating"),
]
for x, y in pairs:
    if x in df.columns and y in df.columns:
        plt.figure()
        plt.scatter(df[x], df[y], alpha=0.4)
        plt.title(f"{y} vs {x}")
        plt.xlabel(x)
        plt.ylabel(y)
        plt.tight_layout()
        plt.show()

## 11. Tendência temporal (Year_num)

In [ ]:
if "Year_num" in df.columns and "IMDB_Rating" in df.columns:
    yearly = df.groupby("Year_num")["IMDB_Rating"].mean().sort_index()
    if len(yearly) > 0:
        r = yearly.rolling(3, min_periods=1).mean()
        plt.figure()
        plt.plot(yearly.index, yearly.values, marker="o", linestyle="-", alpha=0.5)
        plt.plot(r.index, r.values, linestyle="-")
        plt.title("IMDb Rating médio por ano (linha) e média móvel (3)")
        plt.xlabel("Ano")
        plt.ylabel("IMDb Rating (média)")
        plt.tight_layout()
        plt.show()
else:
    print("Sem colunas Year_num/IMDB_Rating suficientes para tendência.")

## 12. (Cuidado com leakage) Gross vs variáveis

In [ ]:
if "Gross_clean" in df.columns:
    for x in ["No_of_Votes", "Meta_score", "IMDB_Rating", "Year_num"]:
        if x in df.columns:
            plt.figure()
            plt.scatter(df[x], df["Gross_clean"], alpha=0.4)
            plt.title(f"Gross_clean vs {x}")
            plt.xlabel(x)
            plt.ylabel("Gross_clean")
            plt.tight_layout()
            plt.show()
else:
    print("Sem 'Gross_clean' para analisar.")

---

### ✅ Checklist rápido
- Colunas padronizadas (`Runtime_min`, `Year_num`, `Gross_clean`, `Genre_main`).
- Conferência de missing e duplicatas.
- Estatísticas descritivas e correlações.
- Distribuições e relações principais.
- Overview (texto) inspecionado por tamanho e exemplos.

> Próximo passo: usar `src/filmlab/train.py` para treinar **genre**, **rating** e **gross**, evitando *leakage*.